In [ ]:
import os
import nltk
import logging
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
import google.generativeai as genai
from google.oauth2 import service_account
from pydantic import BaseModel, Field
from rouge_score import rouge_scorer
from collections import Counter
import re
import sacrebleu
from collections import Counter
from docx import Document as DocxDocument
from tqdm import tqdm
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from bert_score import score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# Download required NLTK resources
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

In [ ]:
csv_path = "../data/question-answers.csv"
try:
    # Load the CSV file with a specified encoding
    data = pd.read_csv(csv_path, encoding="utf-8")
except UnicodeDecodeError:
    # If utf-8 fails, try a different encoding
    data = pd.read_csv(csv_path, encoding="ISO-8859-1")

In [ ]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english")
                 )
# Define text preprocessing function with lemmatization
def preprocess_text(text):
    # 1. Strip whitespace
    text = text.strip()

    # 2. Remove special characters and numbers (optional, depending on requirements)
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # 3. Tokenize the text
    tokens = nltk.word_tokenize(text)

    # 4. Remove stopwords and apply lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tqdm(tokens, desc="Processing tokens") if token.lower() not in stop_words]

    # 5. Join the tokens back into a string
    preprocessed_text = " ".join(lemmatized_tokens)

    return preprocessed_text

In [ ]:
def evaluate_rouge(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    return scorer.score(reference, predicted)


def evaluate_bleu(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    bleu = sacrebleu.corpus_bleu([predicted], [[reference]])
    return bleu.score


def evaluate_f1(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    predicted_tokens = nltk.word_tokenize(preprocess_text(predicted))
    reference_tokens = nltk.word_tokenize(preprocess_text(reference))
    predicted_counts = Counter(predicted_tokens)
    reference_counts = Counter(reference_tokens)

    common_tokens = sum((predicted_counts & reference_counts).values())

    precision = common_tokens / sum(predicted_counts.values()) if predicted_counts else 0
    recall = common_tokens / sum(reference_counts.values()) if reference_counts else 0

    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


def evaluate_bert_score(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    P, R, F1 = score([predicted], [reference], lang="en", verbose=False)
    return F1.mean().item()


# Initialize SentenceTransformer for embedding similarity
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")


def evaluate_embedding_similarity(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    predicted_embedding = sentence_model.encode(predicted)
    reference_embedding = sentence_model.encode(reference)
    similarity = cosine_similarity([predicted_embedding], [reference_embedding])[0][0]
    return similarity


def evaluate_hybrid(predicted, reference):
    rouge = evaluate_rouge(predicted, reference)
    rouge_score = rouge["rougeL"].fmeasure

    bleu_score = evaluate_bleu(predicted, reference)
    f1_score = evaluate_f1(predicted, reference)
    bert_score = evaluate_bert_score(predicted, reference)
    embedding_similarity = evaluate_embedding_similarity(predicted, reference)

    # Define a weighted hybrid score
    hybrid_score = (0.2 * rouge_score) + (0.2 * bleu_score) + (0.2 * f1_score) + (0.2 * bert_score) + (0.2 * embedding_similarity)
    return hybrid_score

In [ ]:
rag_number = input("Please enter the RAG number: ")

In [ ]:
# Add a new column for predicted answers
data[f"RAG {rag_number}"] = ""

# Initialize list to hold the predicted answers
predicted_answers = []

# Initialize lists to hold metric scores
rouge_scores, bleu_scores, f1_scores, bert_scores, embedding_similarities, hybrid_scores = [], [], [], [], [], []

# Initialize min and max values for each metric
rouge_min, rouge_max = float("inf"), float("-inf")
bleu_min, bleu_max = float("inf"), float("-inf")
f1_min, f1_max = float("inf"), float("-inf")
bert_min, bert_max = float("inf"), float("-inf")
embedding_min, embedding_max = float("inf"), float("-inf")

In [ ]:
# First pass to collect metric values and determine min and max for normalization
for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing questions", leave=False):
    question = row["Question"]
    reference_answer = row["Answer"]
    predicted_answer = answer_query(question)
    predicted_answers.append(predicted_answer)

    # Evaluate the metrics
    rouge = evaluate_rouge(predicted_answer, reference_answer)
    bleu = evaluate_bleu(predicted_answer, reference_answer)
    f1 = evaluate_f1(predicted_answer, reference_answer)
    bert_score = evaluate_bert_score(predicted_answer, reference_answer)
    embedding_similarity = evaluate_embedding_similarity(predicted_answer, reference_answer)

    # Update min and max values for each metric
    rouge_scores.append(rouge["rougeL"].fmeasure)
    bleu_scores.append(bleu)
    f1_scores.append(f1)
    bert_scores.append(bert_score)
    embedding_similarities.append(embedding_similarity)

    rouge_min, rouge_max = min(rouge_min, rouge["rougeL"].fmeasure), max(rouge_max, rouge["rougeL"].fmeasure)
    bleu_min, bleu_max = min(bleu_min, bleu), max(bleu_max, bleu)
    f1_min, f1_max = min(f1_min, f1), max(f1_max, f1)
    bert_min, bert_max = min(bert_min, bert_score), max(bert_max, bert_score)
    embedding_min, embedding_max = min(embedding_min, embedding_similarity), max(embedding_max, embedding_similarity)

In [ ]:
# Second pass to normalize the metrics and calculate the hybrid score
for index in tqdm(range(len(data)), desc="Calculating hybrid scores"):
    rouge = rouge_scores[index]
    bleu = bleu_scores[index]
    f1 = f1_scores[index]
    bert_score = bert_scores[index]
    embedding_similarity = embedding_similarities[index]

    # Normalize each metric
    rouge_normalized = (rouge - rouge_min) / (rouge_max - rouge_min) if rouge_max != rouge_min else 0.5
    bleu_normalized = (bleu - bleu_min) / (bleu_max - bleu_min) if bleu_max != bleu_min else 0.5
    f1_normalized = (f1 - f1_min) / (f1_max - f1_min) if f1_max != f1_min else 0.5
    bert_normalized = (bert_score - bert_min) / (bert_max - bert_min) if bert_max != bert_min else 0.5
    embedding_normalized = (embedding_similarity - embedding_min) / (embedding_max - embedding_min) if embedding_max != embedding_min else 0.5

    # Calculate hybrid score using normalized values
    hybrid_score = (0.2 * rouge_normalized) + (0.2 * bleu_normalized) + (0.2 * f1_normalized) + (0.2 * bert_normalized) + (0.2 * embedding_normalized)
    hybrid_scores.append(hybrid_score)

    # Save the predicted answer and hybrid score in the DataFrame
    predicted_answer = predicted_answers[index]
    data.at[index, "RAG 1"] = predicted_answer

    # Print each result
    print(f"[Processing question {index + 1}]:\n")
    print(f"ROUGE-L: {rouge}")
    print(f"BLEU: {bleu}")
    print(f"F1: {f1}")
    print(f"BERT Score: {bert_score}")
    print(f"Embedding Similarity: {embedding_similarity}")
    print(f"Hybrid Score: {hybrid_score}")
    print("-" * 80)

# Save the DataFrame with predictions and hybrid scores to a new CSV file
data.to_csv(f"../data/answers_rag{rag_number}.csv", index=False)

In [ ]:
# Save the evaluation metrics to a CSV file
evaluation_metrics = {
    "ROUGE-L": rouge_scores,
    "BLEU": bleu_scores,
    "F1": f1_scores,
    "BERT Score": bert_scores,
    "Embedding Similarity": embedding_similarities,
    "Hybrid Score": hybrid_scores
}
evaluation_df = pd.DataFrame(evaluation_metrics)

evaluation_df.to_csv(f"../data/rag{rag_number}_evaluation_metrics.csv", index=False)

# Print evaluation summary
print("=== Evaluation Summary ===")
print(f"Average ROUGE-L: {sum(rouge_scores) / len(rouge_scores):.4f}")
print(f"Average BLEU: {sum(bleu_scores) / len(bleu_scores):.2f}")
print(f"Average F1: {sum(f1_scores) / len(f1_scores):.4f}")
print(f"Average BERT Score: {sum(bert_scores) / len(bert_scores):.4f}")
print(f"Average Embedding Similarity: {sum(embedding_similarities) / len(embedding_similarities):.4f}")
print(f"Average Hybrid Score: {sum(hybrid_scores) / len(hybrid_scores):.4f}")